# Определение языка и VK API

В данном задании вам нужно будет:

* используя API Вконтакте, скачать комментарии к первым ста постам из пяти сообществ
* натренировать модель распознавания языков на статьях из Википедии.
* распознать язык всех комментариев, где в тексте есть 10 и более символов, и построить статистику

## VK API

Для подключения к ВКонтакте мы будем использовать VK API. Здесь есть документация к этой библиотеке https://vk-api.readthedocs.io/en/latest/

In [ ]:
#!pip install vk_api
#!pip install sklearn

In [ ]:
import vk_api

# авторизация
vk_session = vk_api.VkApi(login=user, password=password)
vk_session.auth()

vk = vk_session.get_api() # объект с API

Получить записи со страницы можно с помощью метода `wall.get`. Он принимает параметр `domain` — короткое имя пользователя или сообщества — и `count` — количество записей, которое вы хотите получить (максимум — 100). По Список методов для работы со стенами: https://vk.com/dev/wall

Например, вот так можно получить последние две записи с вот этой страницы https://vk.com/futureisnow. Выдача представляет собой словарь, в котором в поле `items` записан список словарей, содержащий информацию о каждой из записи. 

In [ ]:
vk.wall.get(domain="futureisnow", count=2)

С помощью метода `groups.getById` можно получить информацию о сообществе, в том числе его id.

In [ ]:
vk.groups.getById(group_ids="futureisnow")

Скачайте информацию о последних ста записях в следующих пабликах: https://vk.com/futureisnow, https://vk.com/eternalclassic, https://vk.com/ukrlit_memes, https://vk.com/ukrainer_net, https://vk.com/amanzohel, https://vk.com/barg_kurumk_culture

In [ ]:
# список domain'ов, чтобы вам не копировать их самими :)
publics = ["futureisnow",
           "eternalclassic",
           "ukrlit_memes",
           "ukrainer_net",
           "amanzohel",
           "barg_kurumk_culture"]

In [ ]:
items = {} # ключи — это паблики
for i in publics:
    items[i] = {}
for k in items.keys():
    items[k] = vk.wall.get(domain = k, count = 100)


Найдите в документации (https://vk.com/dev/wall) метод для получения комментариев и получите первые сто комментариев каждого поста из выборки для каждого паблика.

In [ ]:
corpora = {}
for k in items.keys():
    i = 0
    while i < 99:
        key = items[k]['items'][i]['id']
        public_id = items[k]['items'][i]['owner_id']
        corpora[key] = vk.wall.getComments(owner_id = public_id, post_id = key, count = 100)
        i += 1

**Задание со звездочкой:** вы могли заметить, что если обращаться к каждому посту отдельно, то все занимает довольно продолжительное время (около пяти минут). Найдите в документации vk_api способ сделать это быстрее.

In [ ]:
corpora = {}

# ваш улучшенный код

## Тренировка моделей

В наших комментариях встречались русский, украинский, английский и бурятский.

In [ ]:
langs = {'ru', 'uk', 'en', 'bxr'}

Скачайте документы, на которых вы будете обучать свои модели. Для наших целей хорошо иметь для каждого языка корпус размером около 50 статей.

In [ ]:
from collections import Counter

import nltk
import wikipedia

def get_texts_for_lang(lang, n=10):
    wiki_content = []
    wikipedia.set_lang(lang)
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
            wiki_content.append("%s\n%s" % (page.title, page.content.replace('=', '')))
        except wikipedia.exceptions.WikipediaException:
            print("Skip %s" % page_name)
    return wiki_content

wiki_texts = {}
for lang in langs:
    wiki_texts[lang] = get_texts_for_lang(lang, 50)
    print(lang, len(wiki_texts[lang]))

Сделайте определялку на частотах слов.

In [ ]:
nltk.download("punkt")

def collect_freqlist(wiki_pages, max_len=100):
    freqlist = Counter()
    for text in wiki_pages:
        for word in nltk.word_tokenize(text.lower()):
            if word.isalpha():
                freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

freq_lists = {}
for lang in langs:
    freq_lists[lang] = collect_freqlist(wiki_texts[lang])
    
def simple_lang_detect(freq_lists, text):
    counts = Counter()
    for lang, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[lang] += int(freq_list[word] > 0)
    return counts.most_common()

Сделайте определялку на символьных энграммах.

In [ ]:
from sklearn import feature_extraction
from sklearn import pipeline
from sklearn import naive_bayes
import numpy as np

clf = pipeline.Pipeline([
    ('vctr', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2), analyzer='char')),
    ('clf', naive_bayes.MultinomialNB())
])


all_texts = []
lang_indices = []
for lang in wiki_texts:
    all_texts.extend(wiki_texts[lang])
    lang_indices.extend([lang]*len(wiki_texts[lang]))
    
clf.fit(np.array(all_texts), np.array(lang_indices))

# Определение языка

Определите язык каждого комментария в каждом паблике с помощью определялки на частотах слов и покажите доли языков среди комментариев для каждого паблика.

In [ ]:
lang_detects_freqs = {}

for k in corpora.keys():
    i = 0
    if len(corpora[k]['items']) >= 1:
        while i < len(corpora[k]['items']):
            if 'text' in corpora[k]['items'][i].keys() and len(corpora[k]['items'][i]['text']) >= 10:
                key = corpora[k]['items'][i]['id']
                lang_detects_freqs[key] = corpora[k]['items'][i]['text']
            i += 1

for v in lang_detects_freqs.values():
    text = str(v)
    language = simple_lang_detect(freq_lists, text)

Сделайте то же самое для определителя на символьных энграммах.

In [ ]:
ng_texts = list(lang_detects_freqs.values())
clf.predict(ng_texts)

Обсудите работу каждого из классификаторов, обсудите ошибки, объясните разницу в результатах.

<Сюда вставьте свой текст (эту ячейку можно отредактировать нажав на нее дважды левой кнопкой мышки и сохранить нажав Ctrl+Enter)>